In [8]:
import os
import pandas as pd
import json
from tqdm import tqdm
from glob import glob


In [33]:
split_dir = "d:/multiagent-disaster-reasoning/data/CrisisMMD_v2.0/crisismmd_datasplit_all"
image_dir ="d:/multiagent-disaster-reasoning/data/CrisisMMD_v2.0/data_image"

informative_files=glob(f"{split_dir}/task_informative_text_img_*.tsv")
humanitarian_files =glob(f"{split_dir}/task_humanitarian_text_img_*.tsv")
damage_files =glob(f"{split_dir}/task_damage_text_img_*.tsv")

informative_df =pd.concat([pd.read_csv(f,sep="\t") for f in informative_files],ignore_index=True)
humanitarian_df =pd.concat([pd.read_csv(f,sep="\t") for f in humanitarian_files],ignore_index=True)
damage_df=pd.concat([pd.read_csv(f,sep="\t") for f in damage_files],ignore_index=True)

merged =informative_df.merge(humanitarian_df, on="tweet_id",suffixes=('_informative', '_humanitarian'))
merged = merged.merge(damage_df, on='tweet_id')

merged.rename(columns={
    'image_id_informative': 'image_id',
    'label_informative': 'informative',
    'label_humanitarian': 'humanitarian',
    'label': 'damage'
},inplace =True)

print("merged data shape:" , merged.shape)
merged.head()


merged data shape: (9969, 22)


,event_name_informative,tweet_id,image_id,tweet_text_informative,image_informative,informative,label_text_informative,label_image_informative,label_text_image_informative,event_name_humanitarian,...,image_humanitarian,humanitarian,label_text_humanitarian,label_image_humanitarian,label_text_image_humanitarian,event_name,image_id,tweet_text,image,damage
0,hurricane_maria,922857566220283904,922857566220283904_0,The gym above subway got destroyed. The cement...,data_image/hurricane_maria/24_10_2017/92285756...,informative,informative,informative,Positive,hurricane_maria,...,data_image/hurricane_maria/24_10_2017/92285756...,infrastructure_and_utility_damage,infrastructure_and_utility_damage,infrastructure_and_utility_damage,Positive,hurricane_maria,922857566220283904_0,The gym above subway got destroyed. The cement...,data_image/hurricane_maria/24_10_2017/92285756...,mild_damage
1,california_wildfires,920571592131915777,920571592131915777_0,Californiaâ€™s Sausalito Fire In Marin Capture...,data_image/california_wildfires/18_10_2017/920...,informative,informative,informative,Positive,california_wildfires,...,data_image/california_wildfires/18_10_2017/920...,infrastructure_and_utility_damage,other_relevant_information,infrastructure_and_utility_damage,Negative,california_wildfires,920571592131915777_0,Californiaâ€™s Sausalito Fire In Marin Capture...,data_image/california_wildfires/18_10_2017/920...,severe_damage
2,hurricane_maria,913157619455426560,913157619455426560_0,RT @FOXLA: PHOTOS: #PuertoRicoNeedsHelpNow str...,data_image/hurricane_maria/27_9_2017/913157619...,informative,informative,informative,Positive,hurricane_maria,...,data_image/hurricane_maria/27_9_2017/913157619...,infrastructure_and_utility_damage,other_relevant_information,infrastructure_and_utility_damage,Negative,hurricane_maria,913157619455426560_0,RT @FOXLA: PHOTOS: #PuertoRicoNeedsHelpNow str...,data_image/hurricane_maria/27_9_2017/913157619...,severe_damage
3,hurricane_harvey,905260640213893121,905260640213893121_0,How you can help Victims of NaturalÂ Disasters...,data_image/hurricane_harvey/6_9_2017/905260640...,informative,informative,informative,Positive,hurricane_harvey,...,data_image/hurricane_harvey/6_9_2017/905260640...,infrastructure_and_utility_damage,rescue_volunteering_or_donation_effort,infrastructure_and_utility_damage,Negative,hurricane_harvey,905260640213893121_0,How you can help Victims of NaturalÂ Disasters...,data_image/hurricane_harvey/6_9_2017/905260640...,mild_damage
4,hurricane_irma,909734028689002497,909734028689002497_0,RT @noaaocean: NOAA continues overflights for ...,data_image/hurricane_irma/18_9_2017/9097340286...,informative,informative,informative,Positive,hurricane_irma,...,data_image/hurricane_irma/18_9_2017/9097340286...,infrastructure_and_utility_damage,other_relevant_information,infrastructure_and_utility_damage,Negative,hurricane_irma,909734028689002497_0,RT @noaaocean: NOAA continues overflights for ...,data_image/hurricane_irma/18_9_2017/9097340286...,severe_damage


In [28]:
merged.columns

Index(['event_name_informative', 'tweet_id', 'image_id',
       'tweet_text_informative', 'image_informative', 'informative',
       'label_text_informative', 'label_image_informative',
       'label_text_image_informative', 'event_name_humanitarian',
       'image_id_humanitarian', 'tweet_text_humanitarian',
       'image_humanitarian', 'humanitarian', 'label_text_humanitarian',
       'label_image_humanitarian', 'label_text_image_humanitarian',
       'event_name', 'image_id', 'tweet_text', 'image', 'damage'],
      dtype='object')

In [34]:
json_dir = "d:/multiagent-disaster-reasoning/data/CrisisMMD_v2.0/json"
disaster_map ={}

for file in os.listdir(json_dir):
    if not file.endswith(".json") or file.startswith("._"): continue
    path = os.path.join(json_dir,file)
    disaster_type =file.replace("_final_data.json", "")
    try:
        df=pd.read_json(path,encoding="utf-8",lines =True)
        for tweet_id in df['id'].astype(str).values:
            disaster_map[tweet_id] = disaster_type
    except Exception as e:
        print(f"skipping {file} due to error: {e}")

merged['tweet_id'] =merged['tweet_id'].astype(str)
merged['disaster_type'] = merged['tweet_id'].map(disaster_map)
merged = merged.dropna(subset=['disaster_type'])

skipping mexico_earthquake_final_data.json due to error: Trailing data


In [37]:
merged["image_name"]=merged["tweet_id"] + "_0.jpg"
merged["image_path"] =merged["image_name"].apply(lambda X:os.path.join(image_dir,X))
merged =merged[merged["image_path"].apply(os.path.exists)]
merged.drop(columns=['image_path'],inplace=True)

fusion_csv_path = os.path.join("d:/multiagent-disaster-reasoning/data/CrisisMMD_v2.0","fusion.csv")
merged.to_csv(fusion_csv_path,index = False)
print("fusion.csv file saved with shape:" , merged.shape)

fusion.csv file saved with shape: (9301, 24)


In [35]:
print("Merged columns:", merged.columns.tolist())


Merged columns: ['event_name_informative', 'tweet_id', 'image_id', 'tweet_text_informative', 'image_informative', 'informative', 'label_text_informative', 'label_image_informative', 'label_text_image_informative', 'event_name_humanitarian', 'image_id_humanitarian', 'tweet_text_humanitarian', 'image_humanitarian', 'humanitarian', 'label_text_humanitarian', 'label_image_humanitarian', 'label_text_image_humanitarian', 'event_name', 'image_id', 'tweet_text', 'image', 'damage', 'disaster_type']


In [38]:
data=pd.read_csv("d:/multiagent-disaster-reasoning/data/fusion.csv")
data.columns

Index(['event_name_informative', 'tweet_id', 'image_id',
       'tweet_text_informative', 'image_informative', 'informative',
       'label_text_informative', 'label_image_informative',
       'label_text_image_informative', 'event_name_humanitarian',
       'image_id_humanitarian', 'tweet_text_humanitarian',
       'image_humanitarian', 'humanitarian', 'label_text_humanitarian',
       'label_image_humanitarian', 'label_text_image_humanitarian',
       'event_name', 'image_id.1', 'tweet_text', 'image', 'damage',
       'disaster_type', 'image_name'],
      dtype='object')